<a href="https://colab.research.google.com/github/jvwk/Product-Sales-Prediction/blob/main/Product_Sales_Prediction_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Sales Prediction Modelling
###Jaco van Wyk
-----------------------


## Data dictionary

Variable | Name	Description
--- | ---
Item_Identifier |	Unique product ID
Item_Weight |	Weight of product
Item_Fat_Content |	Whether the product is low fat or regular
Item_Visibility |	The percentage of total display area of all products in a store allocated to the particular product
Item_Type |	The category to which the product belongs
Item_MRP |	Maximum Retail Price (list price) of the product
Outlet_Identifier |	Unique store ID
Outlet_Establishment_Year |	The year in which store was established
Outlet_Size |	The size of the store in terms of ground area covered
Outlet_Location_Type |	The type of area in which the store is located
Outlet_Type |	Whether the outlet is a grocery store or some sort of supermarket
Item_Outlet_Sales |	Sales of the product in the particular store. This is the target variable to be predicted.

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
from google.colab import drive

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn import set_config
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

set_config(transform_output='pandas')

# Load data directly from url
df = pd.read_csv('/content/drive/MyDrive/CodingDojo/02-MachineLearning/Week06/Data/sales_predictions_2023.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Data cleaning

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
# Drop duplicates
df.duplicated().sum()


0

No duplicates

In [5]:
# Check for inconsistencies in categorical data
cat_cols = df.select_dtypes("object").columns
for i in cat_cols:
  print(f'Values for {i}:')
  print(df[{i}].value_counts())
  print("\n")

Values for Item_Identifier:
Item_Identifier
FDG33              10
FDW13              10
NCL31               9
FDX04               9
NCQ06               9
                   ..
FDN52               1
FDC23               1
FDT35               1
DRF48               1
FDE52               1
Length: 1559, dtype: int64


Values for Item_Fat_Content:
Item_Fat_Content
Low Fat             5089
Regular             2889
LF                   316
reg                  117
low fat              112
dtype: int64


Values for Item_Type:
Item_Type            
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafo

<ipython-input-5-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-5-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-5-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-5-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-5-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(df[{i}].value_counts())
<ipython-input-5-3dfe81ae880d>:5: FutureWarning: Passing a set as an indexer is deprecated and will raise

In [6]:
# Check for unique categorical feature entries
for i in cat_cols:
  print(f'{i}:\n{df[i].nunique()} ({round(df[i].nunique()/len(df)*100,2)}%) unique entries\n')

Item_Identifier:
1559 (18.29%) unique entries

Item_Fat_Content:
5 (0.06%) unique entries

Item_Type:
16 (0.19%) unique entries

Outlet_Identifier:
10 (0.12%) unique entries

Outlet_Size:
3 (0.04%) unique entries

Outlet_Location_Type:
3 (0.04%) unique entries

Outlet_Type:
4 (0.05%) unique entries



No features with 100% unique entries

Exclude categorical features with more than 10 unique values from predictive modelling, but keep for post-modelling analysis

In [9]:
# Check for constant numerical feature entries
num_cols = df.select_dtypes("number").columns
for i in num_cols:
  print(f'{i}:\nmin = {df[i].min()}\nmax = {df[i].max()}\n')

Item_Weight:
min = 4.555
max = 21.35

Item_Visibility:
min = 0.0
max = 0.328390948

Item_MRP:
min = 31.29
max = 266.8884

Outlet_Establishment_Year:
min = 1985
max = 2009

Item_Outlet_Sales:
min = 33.29
max = 13086.9648



No features with constant entries

In [10]:
# Fix inconsistencies in Item_Fat_Content
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("LF", "Low Fat")
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("low fat", "Low Fat")
df["Item_Fat_Content"] = df["Item_Fat_Content"].replace("reg", "Regular")
df["Item_Fat_Content"].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

## Pre-processing

In [11]:
# Assign features (X) and target (y)

# The target we are trying to predict
y = df['Item_Outlet_Sales']
# The features we will use to make the prediction
X = df.drop(columns = ['Item_Outlet_Sales'])
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,NCG06,16.350,Low Fat,0.029565,Household,256.4646,OUT018,2009,Medium,Tier 3,Supermarket Type2
7510,FDV57,15.250,Regular,0.000000,Snack Foods,179.7660,OUT018,2009,Medium,Tier 3,Supermarket Type2
5828,FDM27,12.350,Regular,0.158716,Meat,157.2946,OUT049,1999,Medium,Tier 1,Supermarket Type1
5327,FDG24,7.975,Low Fat,0.014628,Baking Goods,82.3250,OUT035,2004,Small,Tier 2,Supermarket Type1
4810,FDD05,19.350,Low Fat,0.016645,Frozen Foods,120.9098,OUT045,2002,NaN,Tier 2,Supermarket Type1


In [12]:
# Create a preprocessing object to prepare the dataset for Machine Learning

# Create lists of column names for numeric, categorical and ordinal columns
num_cols = num_cols.drop(labels = 'Item_Outlet_Sales')
cat_cols = ['Outlet_Identifier', 'Outlet_Type']
ord_cols = ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type']


In [13]:
# Instantiate numerical preprocessors and pipeline
impute_median = SimpleImputer(strategy='median')
scaler_num = StandardScaler()
num_pipe = make_pipeline(impute_median, scaler_num)
num_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [14]:
# Instantiate nominal preprocessors and pipeline
impute_missing = SimpleImputer(strategy='constant', fill_value = "MISSING")
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_pipe = make_pipeline(impute_missing, ohe_encoder)
ohe_pipe


Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [15]:
# Specifying order of categories for our  Ordinal Qual/Cond Columms
fat_content_order = ['Low Fat','Regular']
outlet_size_order = ['Small','Medium', 'High']
outlet_loc_type_order = ['Tier 1','Tier 2', 'Tier 3']
# Making the list of order lists for OrdinalEncoder
ordinal_category_orders = [fat_content_order, outlet_size_order, outlet_loc_type_order]

# Instantiate ordinal preprocessors and pipeline
impute_mostfreq = SimpleImputer(strategy='most_frequent')
ord_encoder = OrdinalEncoder(categories=ordinal_category_orders)
scaler_ord = StandardScaler()
ord_pipe = make_pipeline(impute_mostfreq, ord_encoder, scaler_ord)
ord_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Low Fat', 'Regular'],
                                            ['Small', 'Medium', 'High'],
                                            ['Tier 1', 'Tier 2', 'Tier 3']])),
                ('standardscaler', StandardScaler())])

In [16]:
# Making a numeric tuple for ColumnTransformer
num_tuple = ('numeric', num_pipe, num_cols)
num_tuple


('numeric',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                 ('standardscaler', StandardScaler())]),
 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
        'Outlet_Establishment_Year'],
       dtype='object'))

In [17]:
# Making a ohe_tuple for ColumnTransformer
ohe_tuple = ('categorical', ohe_pipe, cat_cols)
ohe_tuple


('categorical',
 Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value='MISSING', strategy='constant')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
 ['Outlet_Identifier', 'Outlet_Type'])

In [18]:
# Making an ordinal tuple for ColumnTransformer
ord_tuple = ('ordinal', ord_pipe, ord_cols)
ord_tuple


('ordinal',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                 ('ordinalencoder',
                  OrdinalEncoder(categories=[['Low Fat', 'Regular'],
                                             ['Small', 'Medium', 'High'],
                                             ['Tier 1', 'Tier 2', 'Tier 3']])),
                 ('standardscaler', StandardScaler())]),
 ['Item_Fat_Content', 'Outlet_Size', 'Outlet_Location_Type'])

In [19]:
# Use the tuples to create a ColumnTransformer to preprocess the data; set verbose_feature_names_out to False
col_transformer = ColumnTransformer([num_tuple, ohe_tuple, ord_tuple],
                                    verbose_feature_names_out=False)
col_transformer

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 (...
                                 ['Outlet_Identifier', 'Outlet_Type']),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Low '
                                                                              'Fat',
                                                                              'Regular'],
                                                                             ['Small',
                                                                              'Medium',
                                                                              'High'],
                                                                             ['Tier '
                                                                              '1',
                                                                              'Tier '
                                                                              '2',
                                                                              'Tier '
                                                                              '3']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Fat_Content', 'Outlet_Size',
                                  'Outlet_Location_Type'])],
                  verbose_feature_names_out=False)

In [20]:
# Regression metrics custom function

def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:
    # Print Result with Label and Header
    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics
def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):
  # Get predictions for training data
  y_train_pred = reg.predict(X_train)
  # Call the helper function to obtain regression metrics for training data
  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = reg.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )
  # Store results in a dataframe if ouput_frame is True
  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    # Set the label as the index
    results_df = results_df.set_index('Label')
    # Set index.name to none to get a cleaner looking result
    results_df.index.name=None
    # Return the dataframe
    return results_df.round(3)

## Linear regression

In [30]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg

# Instantiate a Default Model
linreg = LinearRegression()

# Model Pipeline with default preprocessor and default model
linreg_pipe = make_pipeline(col_transformer, linreg)
linreg_pipe

# Fit the model pipeline on the training data only
linreg_pipe.fit(X_train, y_train)

# Call custom function for evaluation
evaluate_regression(linreg_pipe, X_train, y_train, X_test, y_test)


------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 849.475
- MSE = 1,302,203.327
- RMSE = 1,141.141
- R^2 = 0.560

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 802.860
- MSE = 1,188,226.203
- RMSE = 1,090.058
- R^2 = 0.569


The R<sup>2</sup> for both the training and test data is in the mid-50% range.

There does not seem to be overfitting, since the training data does not have a high R<sup>2</sup> value.

There does seem to be underfitting, since the both the training and test data have low R<sup>2</sup> values.

## Random Forest

In [38]:
# Instantiate a Default Model
rf = RandomForestRegressor(random_state = 42)

# Model Pipeline with default preprocessor and default model
rf_pipe = make_pipeline(col_transformer, rf)
rf_pipe

# Fit the model pipeline on the training data only
rf_pipe.fit(X_train, y_train)

# Call custom function for evaluation
evaluate_regression(rf_pipe, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 297.857
- MSE = 184,216.317
- RMSE = 429.204
- R^2 = 0.938

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 769.909
- MSE = 1,213,221.888
- RMSE = 1,101.464
- R^2 = 0.560


The R<sup>2</sup> for the training data is quite high at almost 94%, but low for the test data in the mid-50% range.

There does not seem to be underfitting, since the training data has a high R<sup>2</sup> values, but not the test data.

There does seem to be overfitting given the high R<sup>2</sup> for the training data not being reflected in the R<sup>2</sup> of the test data.



The Random Forest model has a higher R<sup>2</sup> for training data than the Linear Regression model, but a similar outcome is not repeated in the test data. This suggests overfitting.

Both models have similarly low R<sup>2</sup> values for test data (with that of the Linear Regression model being marginally higher).

The MSE of the Linear Regression model is slightly lower (better) than that of the Random Forest model, and (consequently) so is RMSE. The Random Forest model has a lower (better) MAE than its counterpart.

In [39]:
# Obtain list of parameters
rf_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('numeric',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
          'Outlet_Establishment_Year'],
         dtype='object')),
                                   ('categorical',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(fill_value='MISSING',
                                                                   strategy='constant')),
                                                    (...
                                    ['Outlet_Identifier', 'Outlet_Type']),
            

In [40]:
# Define param grid with options to try
params = {'randomforestregressor__max_depth': [None,10,15,20],
          'randomforestregressor__n_estimators':[10,100,150,200],
          'randomforestregressor__min_samples_leaf':[2,3,4],
          'randomforestregressor__max_features':['sqrt','log2',None],
          'randomforestregressor__oob_score':[True,False],
          }

# Instantiate the gridsearch
gridsearch = GridSearchCV(rf_pipe, params, n_jobs=-1, cv = 3, verbose=1)

# Fit the gridsearch on training data
gridsearch.fit(X_train, y_train)

# Obtain the best paramters from the gridsearch
gridsearch.best_params_

Fitting 3 folds for each of 288 candidates, totalling 864 fits


{'randomforestregressor__max_depth': 20,
 'randomforestregressor__max_features': 'sqrt',
 'randomforestregressor__min_samples_leaf': 4,
 'randomforestregressor__n_estimators': 100,
 'randomforestregressor__oob_score': True}

In [41]:
# Define a model with the best parameters already refit on the entire training set
best_rf_grid = gridsearch.best_estimator_

# Evalute the tuned model
evaluate_regression(best_rf_grid, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 590.013
- MSE = 722,614.874
- RMSE = 850.068
- R^2 = 0.756

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 736.868
- MSE = 1,115,236.875
- RMSE = 1,056.048
- R^2 = 0.596


Hyperparameter tuning improved the Random Forest model's performance somewhat: the R<sup>2</sup> improved from 0.560 to 0.596 for the testing data, and the RMSE from 1,101 to 1,056.

## Evaluation

The Random Forest model with hyperparameter tuning is recommended. This recommendation is based on the improved regression metrics of the tuned model, namely the higher R<sep>2</sup> and lower Root Mean Squared Error.


Compare the training vs. test scores and answer the question: to what extent is this model overfit/underfit?

Using this model, around 60% of the variance in item outlet sales is explained by:

- Item_Weight
- Item_Fat_Content
- Item_Visibility
- Item_MRP
- Outlet_Identifier
- Outlet_Establishment_Year
- Outlet_Size
- Outlet_Location_Type
- Outlet_Type

The Mean Absolute Error also saw an improvement in the tuned model compared to the default Random Forest model, with an improvement from to 770 to 737. This metrical is an indication of how far off the mark the output sales value prediction is expected to be from the actual value, on average.

While the R<sup>2</sup> of the training data (0.756) is higher than that of the test data, it is not extremely high. Excessive overfitting is not suspected.